In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()
engine = create_engine(os.getenv("DB_URL"))

df = pd.read_sql("SELECT * FROM employee_attrition_dataset", engine)
df.head()

In [ ]:
from models.base import Base, engine
from models.employee import Employee

Base.metadata.create_all(bind=engine)